In [6]:
import re
import os
import json
import pandas as pd
from collections import Counter
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans
from Levenshtein import distance
from tqdm import tqdm
import numpy as np

merged_dict = {}

# specify the directory
directory = './events_each_country_curated_phase4/EGIRIS/'

# iterate over all files in the directory
find = 0
for filename in os.listdir(directory):
    # check if the file matches the pattern "result_{idx}.json"
    if re.match(r'entity_merge_result_new\d+\.json', filename):
        # construct full file path
        find += 1
        filepath = os.path.join(directory, filename)
        # load the json data from the file
        with open(filepath, 'r') as f:
            llm_res = json.load(f)
            
        # merge this data into the final dictionary
        # This will overwrite existing keys. If you want to handle duplicates differently, you'll need to add logic here.
        merged_dict.update(list(llm_res.values())[0])

FileNotFoundError: [Errno 2] No such file or directory: './events_each_country_curated_phase4/EGIRIS/'

In [2]:
data = pd.read_csv('events_each_country_curated_phase4/EGIRIS_CURATED.csv', sep='\t')

In [3]:
data

,Subject,Relation_id,Relation_choice,Object,Md5,Date
0,France,10,Make public statement,Jews,34ba7f96b18d15029a3b044d2273ef86,20150219
1,House Speaker John Boehner,10,Make public statement,Iran deal,773a4d4039ab0ca354f9b20be74057d5,20150219
2,United States,100,Demand or order,Hamas,276a0608f3526d019a058c43cd91cf07,20150219
3,United States,110,Verbally disapprove,Israel,d3f9a383ef3913ddb61b655b43841ae7,20150219
4,Iran,70,Provide aid,International Atomic Energy Agency (IAEA),30b8df072114e4b9dc304ffb28388bdf,20150219
...,...,...,...,...,...,...
834192,Kremlin spokesman Dmitry Peskov,120,Reject,Ukraine,9592dd1b296aa51f5b2053e8311659e2,20220317
834193,Israel Defense Force,60,Engage in material cooperation,Palestinians,584bc429f77e9e6332704981ba7e691e,20220317
834194,JNS,110,Verbally disapprove,Russia,2c2e6c114c859a16cd6b09d106ee129d,20220317
834195,Russia,140,Engage in political dissent,Israel,2c2e6c114c859a16cd6b09d106ee129d,20220317


In [4]:
entity_counter = Counter(data['Subject'].append(data['Object']))
total_entity_stat = {k: v for k, v in sorted(entity_counter.items(), key=lambda item: item[1], reverse=True)}

In [5]:
reversed_dict = {}

for key, value_list in merged_dict.items():
    for value in value_list:
        reversed_dict[value] = key

In [6]:
with open('./events_each_country_curated_phase4/EGIRIS/first_dict.json', 'w') as f:
    json.dump(reversed_dict, f)

In [7]:
discrete_keys = list(set(total_entity_stat.keys()) - set(reversed_dict))

In [8]:
second_round_key = discrete_keys + list(merged_dict.keys())
print(len(second_round_key))

4716


In [9]:
def remove_strings_with_comma(strings):
    return [string for string in strings if ',' not in string]

In [10]:
second_round_key = remove_strings_with_comma(second_round_key)
print(len(second_round_key))

4566


## use edit distance

In [19]:
# Create the distance matrix
dist_matrix = np.zeros((len(second_round_key), len(second_round_key)))
for i in tqdm(range(len(second_round_key))):
    for j in range(len(second_round_key)):
        dist_matrix[i, j] = distance(second_round_key[i], second_round_key[j])


100%|██████████████████████████████████████████████████████████████████████████████████| 4566/4566 [00:17<00:00, 264.34it/s]


In [5]:
kmeans = KMeans(n_clusters=32, random_state=0)
kmeans.fit(dist_matrix)

NameError: name 'KMeans' is not defined

In [4]:
# Create an empty list for each cluster
clusters = {i: [] for i in range(32)}

# Associate each data point with its cluster
for i, label in enumerate(kmeans.labels_):
    clusters[label].append(second_round_key[i])

NameError: name 'kmeans' is not defined

In [3]:
for cluster_id, entities in clusters.items():
    print(f"Cluster {cluster_id}: {len(entities)}")

NameError: name 'clusters' is not defined

In [2]:
clusters[3]

NameError: name 'clusters' is not defined

## use bert embedding

In [48]:
# Load pretrained model/tokenizer
import torch
from transformers import DistilBertModel, DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [52]:
embeddings = []

for entity in tqdm(second_round_key, desc="Extracting embeddings"):
    # Encode the entity and return a tensor in Pytorch
    inputs = tokenizer(entity, return_tensors="pt")

    # Retrieve the embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # The embeddings are stored in the last hidden state
    cls_embeddings = outputs.last_hidden_state[:, 0, :]

    embeddings.append(cls_embeddings)

Extracting embeddings: 100%|███████████████████████████████████████████████████████████| 4566/4566 [00:39<00:00, 116.95it/s]


In [ ]:
embedding = torch.stack(embeddings).squeeze()

In [53]:
embeddings = torch.stack(embeddings).squeeze()
embeddings = embeddings.numpy()

n_clusters = 32  # change this to the number of clusters you want

# Initialize the KMeans model
kmeans = KMeans(n_clusters=n_clusters, random_state=0)

# Fit the model to your data
kmeans.fit(embeddings)

# Create an empty list for each cluster
clusters_bert = {i: [] for i in range(n_clusters)}

# Associate each data point with its cluster
for i, label in enumerate(kmeans.labels_):
    clusters_bert[label].append(second_round_key[i])
    

In [54]:
for cluster_id, entities in clusters_bert.items():
    print(f"Cluster {cluster_id}: {len(entities)}")

Cluster 0: 124
Cluster 1: 213
Cluster 2: 179
Cluster 3: 83
Cluster 4: 50
Cluster 5: 195
Cluster 6: 257
Cluster 7: 116
Cluster 8: 40
Cluster 9: 157
Cluster 10: 170
Cluster 11: 86
Cluster 12: 160
Cluster 13: 148
Cluster 14: 247
Cluster 15: 14
Cluster 16: 176
Cluster 17: 190
Cluster 18: 123
Cluster 19: 107
Cluster 20: 133
Cluster 21: 319
Cluster 22: 77
Cluster 23: 175
Cluster 24: 118
Cluster 25: 149
Cluster 26: 108
Cluster 27: 142
Cluster 28: 89
Cluster 29: 45
Cluster 30: 256
Cluster 31: 120


In [118]:
clusters_bert[0]

['Boycott Israel',
 'Palestine Action',
 'Support Palestine',
 'Blue and White leader Benny Gantz',
 'Destroy Israel',
 'Armenian genocide',
 'Condemn Israel',
 'Jewish Settlers',
 'Israel Anti-Boycott Act',
 'World Jewish Congress President Ronald Lauder',
 'Express support for Israel',
 'Israeli society',
 'Anti-Semitism',
 'Criticize Israel',
 'Anti-Israel activists',
 'Support for Palestine',
 'Boycott of Israel',
 "Israel's right to exist",
 'Move US embassy to Jerusalem',
 'President Isaac Herzog',
 'J.K. Rowling',
 'Holocaust victims',
 'American Jews',
 'Jewish',
 'Jewish Agency',
 'Jewish groups',
 'United Church of Christ',
 'West Bank settlers',
 'Jewish voters',
 'US Jewish leaders',
 'Jewish students',
 'Jewish organizations',
 'British Jews',
 'Jewish Voice for Peace',
 'Conference of Presidents of Major American Jewish Organizations',
 'West Bank settlement',
 'United Methodist Church',
 'Zionist Organization of America',
 'World Zionist Organization',
 'Jewish National 

## entity filtering

In [80]:
filter_rules = [
    "1. Entity should be a thing with distinct and independent existence.",
    "2. You should removes strings that are statements or policy positions or verbs.",
    "3. There may be some noises in the strings, such as '\nU.S.' or '1. U.S.', but it should still be considered as an entity and you should not clean it.",
    "4. Only output strings that are entities in python list format."
]
filter_prompt_rules = f'You are a helper to determine whether a certain string is an political/economical entity.. Perform entity filtering with the following rules:\n' + '\n'.join(filter_rules) + '\n Given entity name list: \n'

In [119]:
idx = 1
print(filter_prompt_rules + f"{clusters_bert[1]}")

You are a helper to determine whether a certain string is an political/economical entity.. Perform entity filtering with the following rules:
1. Entity should be a thing with distinct and independent existence.
2. You should removes strings that are statements or policy positions or verbs.
3. There may be some noises in the strings, such as '
U.S.' or '1. U.S.', but it should still be considered as an entity and you should not clean it.
4. Only output strings that are entities in python list format.
 Given entity name list: 
['Russia', '\nSaudi Arabia', 'Pakistan', 'Bahrain', 'UAE', 'Kosovo', 'Ukraine', 'Libya', 'Egypt', '\nIsrael', 'Sudan', 'Somalia', 'Serbia', 'Saudi Arabia', 'IRAN', 'Morocco', 'Russia and China', 'Afghanistan', 'Egyptian', 'Iran', 'Israeli', 'Ethiopia', 'Lebanon', 'Israel', 'Yemen', 'Iranian', 'Cyprus', '1. Russia', 'Syria', 'Turkey', 'China', '\nIran', 'Greece', 'Eritrea', 'Azerbaijan', 'Qatar', 'Israeli', 'Iranian', 'Israel', 'Iran', 'Saudi Arabia', 'Russia', 'Chi

In [117]:
tmp = 0
tmp_ent = set()
for k, v in entity_after_merge.items():
    tmp += len(set(v))
    tmp_ent.update(set(v))
print(f"idx: {idx}, \n number of entity berfore filtering: {len(clusters_bert[idx])}, \n \
number of entity after filtering: {len(clusters_filter[idx])}, \n \
number of entity after merging: {len(entity_after_merge)}, \n \
number of entity being considered in merging: {tmp}, \n \
entities that have not been considered in the linking process: {set(clusters_filter[idx]) - tmp_ent}")

idx: 0, 
 number of entity berfore filtering: 124, 
 number of entity after filtering: 116, 
 number of entity after merging: 96, 
 number of entity being considered in merging: 116, 
 entities that have not been considered in the linking process: set()


In [135]:
rules = [
    "1. Only merge entities that EXACTLY refer to the same entity such as PERSON, ORGANIZATION, COUNTRY, etc..",
    "2. The output key should be the entity name after merging, value should be list of original entity names.",
    "3. For those entities that cannot be merged, just output original name: [original name]",
    "4. There may be some noises in the strings, such as '\nU.S.' or '1. U.S.', these cases should be merged but you should not clean the original name.",
    "5. Output in JSON format."
]

prompt_rules = f'Based on the above output. Perform entity linking with the following rules:\n' + '\n'.join(rules)

In [115]:
print(prompt_rules)

Based on the above output. Perform entity linking with the following rules:
1. Only merge entities that EXACTLY refer to the same entity such as PERSON, ORGANIZATION, COUNTRY, etc..
2. The output key should be the entity name after merging, value should be list of original entity names.
3. For those entities that cannot be merged, just output original name: [original name]
4. There may be some noises in the strings, such as '
U.S.' or '1. U.S.', these cases should be mergedm but you should not clean the original name.
5. Output in JSON format.


In [174]:
with open('./events_each_country_curated_phase4/EGIRIS/second_round/entity_filter_res.json', 'w') as json_file:
    json.dump(entity_filter_dict, json_file)

## stat for llm filter and merge

In [177]:
import os

In [178]:
entity_after_merge = {}

directory = './events_each_country_curated_phase4/EGIRIS/second_round/'

# iterate over each file in the directory
for filename in os.listdir(directory):
    # check if the filename starts with "entity_merge_gpt4_" and ends with ".json"
    if filename.startswith('entity_merge_gpt4_') and filename.endswith('.json'):
        # extract the index value
        index = int(filename.split('_')[-1].split('.')[0])
        # read the json file
        with open(os.path.join(directory, filename), 'r') as f:
            # load the json file into a dictionary
            data = json.load(f)
            # add the data to your big dictionary using the index as the key
            entity_after_merge[index] = data

In [185]:
merged_dict_second = {}
for sub_dict in entity_after_merge.values():
    merged_dict_second.update(sub_dict)
    
with open('./events_each_country_curated_phase4/EGIRIS/second_round/merged_dict_second.json', 'w') as json_file:
    json.dump(merged_dict_second, json_file)

##### seperate analysis

In [182]:
for idx in list(range(32)):
    tmp = 0
    tmp_ent = set()
    for k, v in entity_after_merge[idx].items():
        tmp += len(set(v))
        tmp_ent.update(set(v))
    print(f"idx: {idx}, \n number of entity berfore filtering: {len(clusters_bert[idx])}, \n \
    number of entity after filtering: {len(entity_filter_dict[idx])}, \n \
    number of entity after merging: {len(entity_after_merge)}, \n \
    number of entity being considered in merging: {tmp}, \n \
    entities that have not been considered in the linking process: {set(entity_filter_dict[idx]) - tmp_ent}")

idx: 0, 
 number of entity berfore filtering: 124, 
     number of entity after filtering: 116, 
     number of entity after merging: 32, 
     number of entity being considered in merging: 116, 
     entities that have not been considered in the linking process: set()
idx: 1, 
 number of entity berfore filtering: 213, 
     number of entity after filtering: 202, 
     number of entity after merging: 32, 
     number of entity being considered in merging: 172, 
     entities that have not been considered in the linking process: {'Iraq', 'Latvia', 'IRAQ'}
idx: 2, 
 number of entity berfore filtering: 179, 
     number of entity after filtering: 102, 
     number of entity after merging: 32, 
     number of entity being considered in merging: 94, 
     entities that have not been considered in the linking process: {'Movement for Quality Government in Israel', 'Yarmouk refugee camp', 'Palestinian chief negotiator Saeb Erekat', 'Kurdish independence referendum', 'Palestinian Islamic Jihad'

##### all results analysis

In [191]:
tmp_ent = set()
for k, v in entity_filter_dict.items():
    tmp_ent.update(set(v))
filtered_ent = set(second_round_key) - tmp_ent ## this is the entity filtered, print it to check
print(f"\n number of entity berfore filtering: {len(second_round_key)}, \n \
number of entity after filtering: {len(tmp_ent)}, \n \
number of entity after merging: {len(merged_dict_second)}")


 number of entity berfore filtering: 4566, 
 number of entity after filtering: 4150, 
 number of entity after merging: 2962


In [203]:
# show the merging results, shown in descending order of the number of entities being merged
for idx, (k, values) in enumerate(sorted(merged_dict_second.items(), key=lambda x: len(x[1]), reverse=True)):
    print(idx, ":", len(values), "\t", k)
    print("\n".join([x.strip() for x in values]))
    print("\n")
    

0 : 19 	 United States
United States government
United States State Department
United States
The United States
United States of America
United States
3. United States
1. United States
United States
United States (US)
United States Congress
United States Senate
United States Department of State
United States Department of Defense
United States Navy
United States military
United States Air Force
United States Department of Justice
United States


1 : 18 	 Houthi militias in Yemen
Houthi militias in Yemen
Houthi rebels in Yemen
Yemen's Houthi rebels
Yemen Houthi rebels
Al Houthi rebels
Houthi militia
Houthi fighters
Yemeni Houthi rebels
Al Houthi militants
Houthi rebels
Houthi militants
Houthi militias
Yemen's Houthis
Yemen's Houthi group
Yemen Huthi rebels
Iran-aligned Houthi group
Al Houthi militia
Houthi insurgents


2 : 16 	 European Union
European Union
EU states
EU Parliament
EU leaders
EU countries
EU foreign policy chief
European Union leaders
European nations
European states
Euro

325 : 2 	 Ron Wyden
Ron Wyden
Wyden


326 : 2 	 Tammy Duckworth
Tammy Duckworth
Duckworth


327 : 2 	 Ghassan Salame
Ghassan Salame
Salame


328 : 2 	 Eric Garcetti
Eric Garcetti
Garcetti


329 : 2 	 U.S.
U.S.
1. U.S.


330 : 2 	 Saudi rulers
Saudi rulers
Saudi Arab


331 : 2 	 Druze community
Druze community
Druze


332 : 2 	 Bedouin community
Bedouin community
Bedouin


333 : 2 	 Ethiopian community
Ethiopian community
Ethiopian immigrants


334 : 2 	 African migrants
African migrants
African asylum seekers


335 : 2 	 Houthi group
Houthi group
Houthi movement


336 : 2 	 Eritrean man
Eritrean man
Eritrean


337 : 2 	 Nation of Islam
Nation of Islam
Nation of Islam leader Louis Farrakhan


338 : 2 	 Yemeni government forces
Yemeni government forces
Yemeni pro-government forces


339 : 2 	 Kurdistan Regional Government
Kurdistan Regional Government
Kurdish Regional Government (KRG)


340 : 2 	 Islamic State in Iraq and Syria (ISIS)
Islamic State in Iraq and Syria (ISIS)
ISIL


341 : 2

645 : 1 	 Gal Gadot
Gal Gadot


646 : 1 	 Isaac Herzog
Opposition leader Isaac Herzog


647 : 1 	 Yair Lapid
Yair Lapid


648 : 1 	 Gabi Ashkenazi
Gabi Ashkenazi


649 : 1 	 Naftali Bennett
Naftali Bennett


650 : 1 	 Knesset Speaker Yariv Levin
Knesset Speaker Yariv Levin


651 : 1 	 Jerusalem mayor
Jerusalem mayor


652 : 1 	 Roni Alsheich
Israel Police Commissioner Roni Alsheich


653 : 1 	 Moshe Kahlon
Kulanu party leader Moshe Kahlon


654 : 1 	 Ariel Sharon
Ariel Sharon


655 : 1 	 Israeli Finance Minister
Israeli Finance Minister


656 : 1 	 Israeli leader
Israeli leader


657 : 1 	 US Secretary of State Pompeo
US Secretary of State Pompeo


658 : 1 	 Jeremy Corbyn
Jeremy Corbyn


659 : 1 	 Narendra Modi
Narendra Modi


660 : 1 	 Theresa May
Theresa May


661 : 1 	 Sergey Lavrov
Sergey Lavrov


662 : 1 	 Imran Khan
Imran Khan


663 : 1 	 Mohammed bin Salman
Saudi Crown Prince Mohammed bin Salman


664 : 1 	 Vladimir Putin
Vladimir Putin


665 : 1 	 Donald Tusk
Donald Tusk


666 

IRNA - Islamic Republic News Agency


988 : 1 	 Interior Minister
Interior Minister


989 : 1 	 Foreign Ministry
Foreign Ministry


990 : 1 	 Health Minister
Health Minister


991 : 1 	 Defense Ministry
Defense Ministry


992 : 1 	 Security cabinet
Security cabinet


993 : 1 	 Education Minister Naftali Bennett
Education Minister Naftali Bennett


994 : 1 	 Cabinet
Cabinet


995 : 1 	 Guardian Council
Guardian Council


996 : 1 	 Military council
Military council


997 : 1 	 Senate
Senate


998 : 1 	 Committee to Protect Journalists
Committee to Protect Journalists


999 : 1 	 Health Ministry officials
Health Ministry officials


1000 : 1 	 Ministerial Committee for Legislation
Ministerial Committee for Legislation


1001 : 1 	 Local authorities
Local authorities


1002 : 1 	 Members of Congress
Members of Congress


1003 : 1 	 Opposition leader
Opposition leader


1004 : 1 	 Expediency Council
Expediency Council


1005 : 1 	 MEPs
MEPs


1006 : 1 	 Health officials
Health officials


1

US Navy


1337 : 1 	 Gulf allies
Gulf allies


1338 : 1 	 US Navy sailors
US Navy sailors


1339 : 1 	 US warships
US warships


1340 : 1 	 Russian jets
Russian jets


1341 : 1 	 US forces
US forces


1342 : 1 	 US Defense Department
US Defense Department


1343 : 1 	 International Criminal Court
International Criminal Court


1344 : 1 	 Egypt court
Egypt court


1345 : 1 	 Federal judge
Federal judge


1346 : 1 	 US court
US court


1347 : 1 	 German court
German court


1348 : 1 	 ICC judges
ICC judges


1349 : 1 	 High Court
High Court


1350 : 1 	 Court of Cassation
Court of Cassation


1351 : 1 	 Belgian court
Belgian court


1352 : 1 	 US Department of Homeland Security
US Department of Homeland Security


1353 : 1 	 US State Department spokesman Ned Price
US State Department spokesman Ned Price


1354 : 1 	 CIA Director John Brennan
CIA Director John Brennan


1355 : 1 	 US media
US media


1356 : 1 	 Media reports
Media reports


1357 : 1 	 Associated Press
Associated Press


1


1676 : 1 	 Haredim
Haredim


1677 : 1 	 Rabbi
Rabbi


1678 : 1 	 Chabad
Chabad


1679 : 1 	 Ateret Cohanim
Ateret Cohanim


1680 : 1 	 Hebron
Hebron


1681 : 1 	 Nablus
Nablus


1682 : 1 	 Havat Gilad
Havat Gilad


1683 : 1 	 Rabbis
Rabbis


1684 : 1 	 Israelis
Israelis


1685 : 1 	 Sinai
Sinai


1686 : 1 	 Haifa
Haifa


1687 : 1 	 Eliezer Marom
Eliezer Marom


1688 : 1 	 Supreme Court Chief Justice Esther Hayut
Supreme Court Chief Justice Esther Hayut


1689 : 1 	 Jerusalem District Court
Jerusalem District Court


1690 : 1 	 Republican Jewish Coalition
Republican Jewish Coalition


1691 : 1 	 Democratic People's Republic of Korea (DPRK)
Democratic People's Republic of Korea (DPRK)


1692 : 1 	 Democratic Party of Iranian Kurdistan (PDKI)
Democratic Party of Iranian Kurdistan (PDKI)


1693 : 1 	 Saeb Erekat
PLO Secretary General Saeb Erekat


1694 : 1 	 Likud Central Committee
Likud Central Committee


1695 : 1 	 Likud lawmakers
Likud lawmakers


1696 : 1 	 Likud activists
Likud acti

2026 : 1 	 Hamas
Hamas


2027 : 1 	 American
American


2028 : 1 	 Americans
Americans


2029 : 1 	 Lebanon
Lebanon


2030 : 1 	 Iraq
Iraq


2031 : 1 	 Yemen
Yemen


2032 : 1 	 Europe
Europe


2033 : 1 	 Hezbollah
Hezbollah


2034 : 1 	 China
China


2035 : 1 	 Britain
Britain


2036 : 1 	 Sudan
Sudan


2037 : 1 	 Morocco
Morocco


2038 : 1 	 Bahrain
Bahrain


2039 : 1 	 Afghanistan
Afghanistan


2040 : 1 	 Libya
Libya


2041 : 1 	 Ukraine
Ukraine


2042 : 1 	 Nigeria
Nigeria


2043 : 1 	 Ethiopia
Ethiopia


2044 : 1 	 Niger
Niger


2045 : 1 	 Pakistan
Pakistan


2046 : 1 	 Mexico
Mexico


2047 : 1 	 Brazil
Brazil


2048 : 1 	 Argentina
Argentina


2049 : 1 	 Chile
Chile


2050 : 1 	 Venezuela
Venezuela


2051 : 1 	 Algeria
Algeria


2052 : 1 	 Tunisia
Tunisia


2053 : 1 	 Qatar
Qatar


2054 : 1 	 Oman
Oman


2055 : 1 	 Kuwait
Kuwait


2056 : 1 	 Saudi
Saudi


2057 : 1 	 West Jerusalem
West Jerusalem


2058 : 1 	 Erbil
Erbil


2059 : 1 	 Mosul
Mosul


2060 : 1 	 Sanaa
Sanaa


2061 : 1 

2354 : 1 	 Lawyers
Lawyers


2355 : 1 	 Researchers
Researchers


2356 : 1 	 Foreigners
Foreigners


2357 : 1 	 Security Chaos
Security Chaos


2358 : 1 	 Thousands of people
Thousands of people


2359 : 1 	 Vaccines
Vaccines


2360 : 1 	 Jordanian
Jordanian


2361 : 1 	 Students
Students


2362 : 1 	 Children
Children


2363 : 1 	 Businessman
Businessman


2364 : 1 	 Journalist
Journalist


2365 : 1 	 Settlements
Settlements


2366 : 1 	 Soldiers
Soldiers


2367 : 1 	 Settler
Settler


2368 : 1 	 Suspect
Suspect


2369 : 1 	 Regime
Regime


2370 : 1 	 Donors
Donors


2371 : 1 	 People
People


2372 : 1 	 Immigrant
Immigrant


2373 : 1 	 Unspecified
Unspecified


2374 : 1 	 Academics
Academics


2375 : 1 	 Voters
Voters


2376 : 1 	 McCain
McCain


2377 : 1 	 Supporters
Supporters


2378 : 1 	 Emirates
Emirates


2379 : 1 	 Telegram
Telegram


2380 : 1 	 Investors
Investors


2381 : 1 	 Resistance
Resistance


2382 : 1 	 Europeans
Europeans


2383 : 1 	 Soldier
Soldier


2384 : 1 	 6 P

2672 : 1 	 Colombia
Colombia


2673 : 1 	 Brunei
Brunei


2674 : 1 	 Cambodia
Cambodia


2675 : 1 	 Tanzania
Tanzania


2676 : 1 	 Dominican Republic
Dominican Republic


2677 : 1 	 Angola
Angola


2678 : 1 	 Namibia
Namibia


2679 : 1 	 Uzbekistan
Uzbekistan


2680 : 1 	 South Sudan
South Sudan


2681 : 1 	 Mali
Mali


2682 : 1 	 Malta
Malta


2683 : 1 	 Bangladesh
Bangladesh


2684 : 1 	 Italy
Italy


2685 : 1 	 Persian Gulf
Persian Gulf


2686 : 1 	 Tatarstan
Tatarstan


2687 : 1 	 Bosnia and Herzegovina
Bosnia and Herzegovina


2688 : 1 	 Djibouti
Djibouti


2689 : 1 	 Jordanian counterpart Ayman Safadi
Jordanian counterpart Ayman Safadi


2690 : 1 	 Turkish FM Mevlut Cavusoglu
Turkish FM Mevlut Cavusoglu


2691 : 1 	 US Congresswoman Ilhan Omar
US Congresswoman Ilhan Omar


2692 : 1 	 Prime Minister Imran Khan
Prime Minister Imran Khan


2693 : 1 	 Prime Minister Ibrahim Mehleb
Prime Minister Ibrahim Mehleb


2694 : 1 	 Morocco king Mohammed VI
Morocco king Mohammed VI


2695 : 1 

In [193]:
reversed_dict = {}

for key, value_list in merged_dict_second.items():
    for value in value_list:
        reversed_dict[value] = key

In [194]:
with open('./events_each_country_curated_phase4/EGIRIS/second_round/reversed_dict_second.json', 'w') as json_file:
    json.dump(reversed_dict, json_file)